# Tools for NLP

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("nlp").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 17:39:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java culd use case slasses"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [5]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java culd ...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [9]:
tokenizer = Tokenizer(inputCol = "sentence", outputCol = "words")

In [10]:
words_data = tokenizer.transform(sentenceData)

In [11]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java culd ...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [12]:
words_data.show(truncate = False)

+-----+-----------------------------------+-----------------------------------------+
|label|sentence                           |words                                    |
+-----+-----------------------------------+-----------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]             |
|0.0  |I wish Java culd use case slasses  |[i, wish, java, culd, use, case, slasses]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat]|
+-----+-----------------------------------+-----------------------------------------+



In [13]:
hashing_tf = HashingTF(inputCol = "words", outputCol = "rawFeatures")

In [14]:
featurized_data = hashing_tf.transform(words_data)

In [15]:
idf = IDF(inputCol = "rawFeatures", outputCol = "features")

In [18]:
idf_model = idf.fit(featurized_data)

In [19]:
rescaled_data = idf_model.transform(featurized_data)

In [20]:
rescaled_data.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



22/05/02 17:55:16 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [21]:
rescaled_data.select("label", "features").show(truncate = False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,94380,98717,192310,238208],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

22/05/02 17:56:06 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


# Vectorizer/ vectors of word counts/ tokenization

In [22]:
from pyspark.ml.feature import CountVectorizer

In [23]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [24]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [26]:
cv = CountVectorizer(inputCol = "words", outputCol = "features", vocabSize = 3, minDF = 2.0)

In [27]:
model = cv.fit(df)

In [28]:
result = model.transform(df)

In [29]:
result.show(truncate = False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

